match_details
a row for every players performance in a match
matches
a row for every match
medals_matches_players
a row for every medal type a player gets in a match
medals
a row for every medal type
Your goal is to make the following things happen:

Build a Spark job that
Disabled automatic broadcast join with spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
Explicitly broadcast JOINs medals and maps
Bucket join match_details, matches, and medal_matches_players on match_id with 16 buckets
Aggregate the joined data frame to figure out questions like:
Which player averages the most kills per game?
Which playlist gets played the most?
Which map gets played the most?
Which map do players get the most Killing Spree medals on?
With the aggregated data set
Try different .sortWithinPartitions to see which has the smallest data size (hint: playlists and maps are both very low cardinality)
Save these as .py files and submit them this way!



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast
import time


def create_spark_session():
    """Create and configure Spark session with optimizations"""
    spark = SparkSession.builder \
        .appName("Spark SQL NBA game analytics") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.sql.sources.bucketing.enabled", "true") \
        .config("spark.sql.sources.bucketing.autoBucketedScan.enabled", "true") \
        .getOrCreate()
    
    # Disable automatic broadcast join threshold as required
    spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
    
    return spark

df_game_details = spark.read.jdbc(url=url, table="game_details", properties=properties)





In [ ]:
df_maps = spark.read.csv(
    "Echooed/data-engineer-handbook/bootcamp/materials/3-spark-fundamentals/data/maps.csv",
    header=True,
    inferSchema=True
)

df_maps.show(5)

In [43]:
def load_all_data(spark, url, properties):
    """Load all required datasets"""
    
    print("Loading all datasets...")
    
    # Load NBA data from JDBC
    print("Loading NBA game details from database...")
    df_game_details = spark.read.jdbc(url=url, table="game_details", properties=properties)
    
    # Load gaming datasets from CSV files
    print("Loading gaming datasets from CSV files...")
    
    # Load maps data
    df_maps = spark.read.csv(
        "/home/dataspiro/data-engineer-handbook/bootcamp/materials/3-spark-fundamentals/data/maps.csv",
        header=True,
        inferSchema=True
    )
    
    # Load other gaming datasets (update paths as needed)
    try:
        df_match_details = spark.read.csv(
            "/home/dataspiro/data-engineer-handbook/bootcamp/materials/3-spark-fundamentals/data/match_details.csv",
            header=True,
            inferSchema=True
        )
        
        df_matches = spark.read.csv(
            "/home/dataspiro/data-engineer-handbook/bootcamp/materials/3-spark-fundamentals/data/matches.csv",
            header=True,
            inferSchema=True
        )
        
        df_medals_matches_players = spark.read.csv(
            "/home/dataspiro/data-engineer-handbook/bootcamp/materials/3-spark-fundamentals/data/medals_matches_players.csv",
            header=True,
            inferSchema=True
        )
        
        df_medals = spark.read.csv(
            "/home/dataspiro/data-engineer-handbook/bootcamp/materials/3-spark-fundamentals/data/medals.csv",
            header=True,
            inferSchema=True
        )
        
        print("All gaming datasets loaded successfully!")
        gaming_data_available = True
        
    except Exception as e:
        print(f"Gaming datasets not available: {e}")
        print("Will proceed with NBA analysis only")
        gaming_data_available = False
        df_match_details = None
        df_matches = None
        df_medals_matches_players = None
        df_medals = None
    
    return {
        'game_details': df_game_details,
        'maps': df_maps,
        'match_details': df_match_details,
        'matches': df_matches,
        'medals_matches_players': df_medals_matches_players,
        'medals': df_medals,
        'gaming_data_available': gaming_data_available
    }

In [ ]:

df_test = spark.read.csv(
    "/home/dataspiro/data-engineer-handbook/bootcamp/materials/3-spark-fundamentals/data/match_details.csv",
    header=True,
    inferSchema=True
)
df_test.show(5)


# df_match_details = spark.read.csv(
#             "Echooed/data-engineer-handbook/bootcamp/materials/3-spark-fundamentals/data/match_details.csv",
#             header=True,
#             inferSchema=True)

In [ ]:
def create_spark_session():
    """Create and configure Spark session with optimizations"""
    spark = SparkSession.builder \
        .appName("Spark SQL NBA game analytics") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.sql.sources.bucketing.enabled", "true") \
        .config("spark.sql.sources.bucketing.autoBucketedScan.enabled", "true") \
        .getOrCreate()
    
    # Disable automatic broadcast join threshold as required
    spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
    
    return spark


spark = SparkSession.builder \
    .appName("Spark SQL NBA game analytics") \
    .config("spark.jars", "/home/your-user/jars/postgresql-42.7.3.jar") \
    .getOrCreate()


25/08/22 10:48:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
# Run this to see what columns you have
print("Your NBA data columns:")
print(df_game_details.columns)
df_game_details.printSchema()
df_game_details.show(5)

In [ ]:
def setup_tables_and_optimizations(spark):
    """Set up NBA tables with proper optimizations using Spark SQL"""
    
    print("Setting up NBA database and tables...")
    
    # Create database
    spark.sql("CREATE DATABASE IF NOT EXISTS nba_analysis")
    spark.sql("USE nba_analysis")
    
    # Create game_details table with bucketing
    df_game_details.write \
        .mode("overwrite") \
        .bucketBy(16, "game_id") \
        .sortBy("game_id", "player_id") \
        .saveAsTable("bucketed_game_details")
    

In [36]:
# 1. Initialize Spark
spark = create_spark_session()

# 2. Define Postgres connection info
url = "jdbc:postgresql://localhost:5434/postgres"
properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

In [ ]:
# test if the jar is loaded

jars = spark.sparkContext._jsc.sc().listJars()
print(jars)


In [ ]:
# Create Spark session with PostgreSQL JDBC driver
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("NBA Database Connection") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.3") \
    .getOrCreate()

print("NBA tables created successfully!")



In [ ]:
# Basic data validation
total_records = spark.sql("SELECT COUNT(*) as total FROM bucketed_game_details").collect()[0]['total']
total_games = spark.sql("SELECT COUNT(DISTINCT game_id) as games FROM bucketed_game_details").collect()[0]['games']
total_players = spark.sql("SELECT COUNT(DISTINCT player_id) as players FROM bucketed_game_details").collect()[0]['players']
    
print(f"Data loaded: {total_records:,} records, {total_games:,} games, {total_players:,} unique players")

In [1]:
# Load table
# df = spark.read.jdbc(url=url, table="game_details", properties=properties)

df_match_details

NameError: name 'df_match_details' is not defined

In [42]:
import os
print(os.path.exists("/home/dataspiro/data-engineer-handbook/bootcamp/materials/3-spark-fundamentals/data/match_details.csv"))


True


In [3]:
# Example: write Spark DataFrame to PostgreSQL
df_matches.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5434/postgres") \
    .option("dbtable", "matches") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .mode("append")  \
    .save()


NameError: name 'df_matches' is not defined